### Imports and setup

In [2]:
# %pip install git+https://github.com/GTSF-Quantitative-Sector/sec_parser.git python-dotenv tqdm pickle matplotlib pandas

In [3]:
from sec import constants, stock, lookups, processor
import pandas as pd
from datetime import date, timedelta
from typing import Iterator
import asyncio
from tqdm import tqdm
import pickle
import os
import dotenv
import matplotlib.pyplot as plt

In [4]:
dotenv.load_dotenv()
API_KEY = os.getenv("POLYGON_API_KEY")

In [5]:
constants.set_polygon_key(API_KEY)
pd.set_option("display.max_rows", None)
pd.set_option("display.max_columns", None)

Run the below cells to get the most recent financials data in SEC parser. Should be run every month or two.

In [6]:
processor.download_sec_data()

Data not stale. Skipping download. To override, set force_update=True or change max_stale_days.


### Data collection

In [7]:
async def gather_with_concurrency(n, *coros):
    semaphore = asyncio.Semaphore(n)
    async def sem_coro(coro):
        async with semaphore:
            return await coro
    return await asyncio.gather(*(sem_coro(c) for c in coros), return_exceptions=True)

In [8]:
def try_get_value(value_func):
    try:
        return value_func()
    except:
        return None

In [9]:
async def get_data(stocks: dict, query_date: str = None) -> dict:
    """
    Gather all the data needed for the embeddings for each given stock.

    Args:
        stocks (dict): Dictionary of stocks to gather data for: {ticker (str): stock (stock.Stock)}
        query_date (str, optional): Date in YYYY-MM-DD format. Defaults to None, which gets the latest metric.
    
    Returns:
        dict: Dictionary of data for each stock: {ticker (str): data (dict)}
    """

    if query_date is None:
        query_date = date.today().strftime("%Y-%m-%d")

    # get price and rsi for each stock
    price_coros, rsi_coros = [], []
    for ticker, s in stocks.items():
        price_coros.append(s.get_price(query_date))
        rsi_coros.append(s.get_rsi(query_date))
    results = await gather_with_concurrency(50, *price_coros, *rsi_coros)
    price_dict = {ticker: price for ticker, price in zip(stocks.keys(), results[: len(stocks)])}
    rsi_dict = {ticker: rsi for ticker, rsi in zip(stocks.keys(), results[len(stocks) :])}

    data = {}
    for ticker, s in stocks.items():
        try:
            price = price_dict[ticker]
            shares = s.get_shares_outstanding(query_date)
            market_cap = shares * price
        except:
            price = None
            market_cap = None

        # get price to book
        try:
            book_val = s.get_book_value(query_date)
            bvps = book_val / shares
            pb_ratio = price / bvps
        except:
            pb_ratio = None

        # get price to earnings
        try:
            net_income = s.get_net_income(query_date)
            try:
                preferred_div = s.get_preferred_dividends(query_date)
            except:
                preferred_div = 0
            eps = (net_income - preferred_div) / shares
            pe_ratio = price / eps
        except:
            pe_ratio = None

        # get price to sales
        try:
            revenue = s.get_revenue(query_date)
            sps = revenue / shares
            ps_ratio = price / sps
        except:
            ps_ratio = None

        # get price to cash flow
        try:
            ocf = s.get_operating_cash_flow(query_date)
            cfps = ocf / shares
            pcf_ratio = price / cfps
        except:
            pcf_ratio = None

        # get ev to ebitda
        try:
            debt = s.get_total_debt(query_date)
            cash = s.get_cash(query_date)
            ev = market_cap + debt - cash
            ebitda = s.get_ebitda(query_date)
            ev_ebitda = ev / ebitda
        except:
            ev_ebitda = None

        # get shareholder yield
        try:
            cash_dividends = s.get_cash_dividends(query_date)
        except:
            cash_dividends = 0
        try:
            share_repurchases = s.get_share_repurchases(query_date)
        except:
            share_repurchases = 0
        try:
            share_issuances = s.get_share_issuances(query_date)
        except:
            share_issuances = 0
        try:
            debt_paydown = s.get_debt_paydown(query_date)
        except:
            debt_paydown = 0
        try:
            debt_issuance = s.get_debt_issuance(query_date)
        except:
            debt_issuance = 0

        try:
            shareholder_yield = (
                cash_dividends
                + share_repurchases
                - share_issuances
                + debt_paydown
                - debt_issuance
            ) / market_cap
        except:
            shareholder_yield = None

        try:
            wacc = s.get_wacc(query_date)
        except Exception:
            wacc = None
            
        rsi = rsi_dict[ticker]
        industry = s.industry

        data[ticker] = {
            "pb_ratio": pb_ratio,
            "pe_ratio": pe_ratio,
            "ps_ratio": ps_ratio,
            "pcf_ratio": pcf_ratio,
            "ev_ebitda": ev_ebitda,
            "shareholder_yield": shareholder_yield,
            "rsi": rsi,
            "price": price,
            "industry": industry,
            "wacc": wacc,
        }

    return data

### Data Cleaning Logic

In [10]:
def clean_data(data: dict, universe: list) -> tuple[pd.DataFrame, float]:
    """
    Handle data corrections, removal of na values, etc..

    Args:
        data (dict): Dictionary of data for each stock: {ticker (str): data (dict)}
        universe (list): Set of tickers to rank (in case data contains more stocks than the universe).

    Returns:
        tuple[pd.DataFrame, float]: Tuple of the ranked DataFrame and the data coverage percentage.
    """
    
    df = pd.DataFrame(data).T
    available_tickers = df.index.intersection(universe)
    df = df.loc[available_tickers]

    for column in ["pb_ratio", "pe_ratio", "ps_ratio", "pcf_ratio", "ev_ebitda", "shareholder_yield", "rsi", "price", "wacc"]:
        df[column] = pd.to_numeric(df[column], errors='coerce')

    # drop rows with nan values, determine coverage percentage
    df = df.dropna()
    coverage = len(df) / len(universe)

    # sort by rsi (high to low)
    df = df[df["rsi"].apply(lambda x: isinstance(x, float))]
    df = df.sort_values("rsi", ascending=False)

    return df, coverage

### Run Data Collection for Current Date

In [11]:
run_for_current = False

In [12]:
if run_for_current:
    stocks = {}
    failed = set()
    sp500 = lookups.get_sp500_tickers()
    for ticker in sp500:
        try:
            stocks[ticker] = stock.Stock(ticker)
        except:
            failed.add(ticker)
            continue

In [13]:
if run_for_current:
    vc2_data = await get_data(stocks)
    df, coverage = clean_data(vc2_data, sp500)
    print(coverage)
    print(df)

### Gather Historical Data

In [14]:
def daterange(start_date: str, end_date: str, interval: int = 1) -> Iterator[date]:
    """
    Generate a range of dates between start_date and end_date.
    
    Args:
        start_date (str): Start date in YYYY-MM-DD format.
        end_date (str): End date in YYYY-MM-DD format.
        interval (int, optional): Interval between dates in days. Defaults to 1.
    
    Returns:
        Iterator[str]: Iterator of dates in string format between start_date and end_date.
    """

    start_date = date.fromisoformat(start_date)
    end_date = date.fromisoformat(end_date)
    delta = end_date - start_date
    for i in range(0, delta.days + 1, interval):
        yield str(start_date + timedelta(days=i))

In [15]:
def get_sp500_universe(start_date: str, end_date: str, interval: int) -> set:
    # get all tickers in the S&P 500 between start_date and end_date
    all_sp500_tickers = set()
    for d in daterange(start_date, end_date, interval):
        all_sp500_tickers.update(lookups.get_sp500_tickers(d))

    return all_sp500_tickers

In [16]:
def process_and_get_stocks(start_date: str, end_date: str, interval: int) -> dict:
    # initialize stocks dictionary: {ticker: stock.Stock}
    stocks = {}
    for ticker in get_sp500_universe(start_date, end_date, interval):
        try:
            processor.process_sec_json(ticker)
            stocks[ticker] = stock.Stock(ticker)
        except Exception as e:
            print(f"Error getting stock data for {ticker}: {e}")

    return stocks

In [17]:
def industry_encoding(data: dict):
    """
    Encodes industry value as a one-hot encoding in dataset.

    Args:
        data (dict): Dictionary of data for each stock at each timepoint: {date (str) : {ticker (str): data (dict)}}

    Returns:
        dict: Dictionary of data for each stock at each timepoint with industry one-hot encoding: {date (str) : {ticker (str): data (dict)}}
    """

    industries = set(d["industry"] for date_data in data.values() for d in date_data.values())
    industry_dict = {industry: i for i, industry in enumerate(sorted(industries))}

    for date_data in data.values():
        for ticker, metrics in date_data.items():
            industry_encoding = [0] * len(industries)
            industry_encoding[industry_dict[metrics["industry"]]] = 1
            date_data[ticker]["industry_encoded"] = industry_encoding

    return data

In [18]:
async def gather_historical_data(stocks: dict, start_date: str, end_date: str, interval: int) -> dict:
    """
    Gather data for all stocks in the S&P 500 between start_date and end_date.

    Args:
        stocks (dict): Dictionary of stocks to gather data for: {ticker (str): stock (stock.Stock)}
        start_date (str): Start date of the backtest.
        end_date (str): End date of the backtest.
        interval (int): Interval between dates in days.

    Returns:
        dict: {str(date): {ticker: {data}}}
    """
    # get all data for all stocks in the S&P 500 between start_date and end_date
    # {str(date): {ticker: {data}}
    data = {}
    for d in tqdm(daterange(start_date, end_date, interval)):
        data[d] = await get_data(stocks, d)

    return data

### Historical Data Gathering Results

In [19]:
BACKTEST_DIR = "saved_data"

def save_backtest_data(data: dict, start_date: str, end_date: str, interval: int, save_dir: str = BACKTEST_DIR):
    os.makedirs(save_dir, exist_ok=True)
    with open(os.path.join(save_dir, f"{start_date}_{end_date}_{interval}.pickle"), "wb") as f:
        pickle.dump(data, f)

In [20]:
GATHER_START_DATE_6M = "2023-07-01"
GATHER_START_DATE_1Y = "2023-01-01"
GATHER_START_DATE_2Y = "2022-01-01"
GATHER_START_DATE_10Y = "2013-01-01"
GATHER_END_DATE = "2024-01-01"
GATHER_INTERVAL_MONTHLY, GATHER_INTERVAL_WEEKLY, GATHER_INTERVAL_DAILY = 30, 7, 1

In [21]:
should_reprocess_data = False

In [22]:
if should_reprocess_data:
    stocks = process_and_get_stocks(GATHER_START_DATE_10Y, GATHER_END_DATE, 1)

In [23]:
if should_reprocess_data:
    # Get monthly backtest data
    monthly = await gather_historical_data(stocks, GATHER_START_DATE_10Y, GATHER_END_DATE, GATHER_INTERVAL_MONTHLY)
    save_backtest_data(monthly, GATHER_START_DATE_10Y, GATHER_END_DATE, GATHER_INTERVAL_MONTHLY)

In [24]:
if should_reprocess_data:
    # get weekly backtest data
    weekly = await gather_historical_data(stocks, GATHER_START_DATE_1Y, GATHER_END_DATE, GATHER_INTERVAL_WEEKLY)
    save_backtest_data(weekly, GATHER_START_DATE_1Y, GATHER_END_DATE, GATHER_INTERVAL_WEEKLY)

In [25]:
if should_reprocess_data:
    # get daily backtest data
    daily = await gather_historical_data(stocks, GATHER_START_DATE_6M, GATHER_END_DATE, GATHER_INTERVAL_DAILY)
    save_backtest_data(daily, GATHER_START_DATE_6M, GATHER_END_DATE, GATHER_INTERVAL_DAILY)

In [26]:
if should_reprocess_data:
    # Get daily backtest data for 2Y
    daily = await gather_historical_data(stocks, GATHER_START_DATE_2Y, GATHER_END_DATE, GATHER_INTERVAL_DAILY)
    save_backtest_data(daily, GATHER_START_DATE_2Y, GATHER_END_DATE, GATHER_INTERVAL_DAILY)

In [27]:
def load_backtest_data(start_date: str, end_date: str, interval: int, save_dir: str = BACKTEST_DIR):
    with open(os.path.join(save_dir, f"{start_date}_{end_date}_{interval}.pickle"), "rb") as f:
        return pickle.load(f)

In [76]:
weekly = load_backtest_data(GATHER_START_DATE_10Y, GATHER_END_DATE, GATHER_INTERVAL_WEEKLY)
daily = load_backtest_data(GATHER_START_DATE_2Y, GATHER_END_DATE, GATHER_INTERVAL_DAILY)

# Cut off weekly data at GATHER_START_DATE_2Y, since I originally gathered for all years
weekly = {k: v for k, v in weekly.items() if k < GATHER_START_DATE_2Y}

# Combine results into one dataset, where dataset is {date (str) : {ticker (str): data (dict)}}
dataset = {**weekly, **daily}
dataset = industry_encoding(dataset)

dataset_df = pd.DataFrame.from_dict(
    {(i, j): dataset[i][j] for i in dataset.keys() for j in dataset[i].keys()},
    orient="index",
)

574
470


In [6]:
df.index = pd.MultiIndex.from_tuples([(pd.to_datetime(date), ticker) for date, ticker in df.index], names=["Date", "Ticker"])
df.sort_index(level=["Date", "Ticker"], inplace=True)

for column in ["pb_ratio", "pe_ratio", "ps_ratio", "pcf_ratio", "ev_ebitda", "shareholder_yield", "rsi", "price", "wacc"]:
    df[column] = pd.to_numeric(df[column], errors='coerce')

df.dropna(inplace=True)

# Up to GATHER_START_DATE_2Y, calculate 7day price change using previous price for eack ticker + date combo
GATHER_START_DATE_2Y_dt = pd.to_datetime(GATHER_START_DATE_2Y)
weekly_df = df[df.index.get_level_values('Date') <= GATHER_START_DATE_2Y_dt].copy()
weekly_df["7d_price_change"] = weekly_df.groupby("Ticker")["price"].pct_change(1)
daily_df = df[df.index.get_level_values('Date') >= GATHER_START_DATE_2Y_dt - pd.DateOffset(days=7)].copy()
daily_df["7d_price_change"] = daily_df.groupby("Ticker")["price"].pct_change(7)
daily_df = daily_df[daily_df.index.get_level_values('Date') > GATHER_START_DATE_2Y_dt] # remove extra rows from weekly_df

df = pd.concat([weekly_df, daily_df])
df["7d_price_change"].fillna(0, inplace=True) # Set to 0 for values that can't calculate change

df.index.names = ["Date", "Ticker"]
df["unix_time"] = df.index.get_level_values("Date").map(lambda x: x.timestamp())

# Rearrange columns for better readability
df = df[["unix_time", "price", "7d_price_change", "rsi", "pb_ratio", "pe_ratio", "ps_ratio", "pcf_ratio", "ev_ebitda", "shareholder_yield", "wacc", "industry", "industry_encoded"]]

NameError: name 'df' is not defined

In [78]:
df.to_csv(f"{BACKTEST_DIR}/dataset.csv")

In [13]:
#import pandas as pd
datasetDF = pd.read_csv("saved_data/dataset.csv")
embeddingDF = pd.read_csv("saved_data/embeddings.csv")
datasetDF["e_vector_0"] = embeddingDF["0"]
datasetDF["e_vector_1"] = embeddingDF["1"]
datasetDF.set_index(["Date", "Ticker"], inplace=True)
datasetDF.drop(columns=["industry_encoded"], inplace=True)
datasetDF.to_csv("saved_data/dynamo.csv")

In [33]:
priceDF = pd.DataFrame(datasetDF)
priceDF.drop(columns=["unix_time", "7d_price_change", "rsi", "pb_ratio", "pe_ratio", "ps_ratio", "pcf_ratio", "ev_ebitda", "shareholder_yield", "wacc", "industry", "e_vector_0", "e_vector_1"], inplace=True)
priceDF.reset_index(inplace=True)
groupedDF = priceDF.groupby("Ticker").agg({"price": list, "Date": list}).reset_index()
groupedDF.set_index("Ticker", inplace=True)
groupedDF.to_csv("saved_data/prices.csv")